## 1. Environment Setup & Package Installation

Run this cell to install all required packages with CUDA support.

In [ ]:
# ========== CUDA Environment Setup ==========
# Run these commands in terminal before starting Jupyter:

# 1. Create virtual environment
# python3.11 -m venv niraakshan_cuda_env
# source niraakshan_cuda_env/bin/activate

# 2. Install CUDA-enabled packages
!pip install --upgrade pip setuptools wheel

# Core packages
!pip install opencv-contrib-python==4.8.1.78
!pip install numpy==1.24.3
!pip install Pillow==10.1.0

# CUDA-accelerated face recognition
# Install dlib with CUDA support (pre-built or compile from source)
!pip install face-recognition==1.3.0
# For CUDA-enabled dlib, compile from source:
# git clone https://github.com/davisking/dlib.git
# cd dlib
# mkdir build && cd build
# cmake .. -DDLIB_USE_CUDA=1 -DUSE_AVX_INSTRUCTIONS=1
# cmake --build . --config Release
# cd ..
# python setup.py install

# MediaPipe with GPU support
!pip install mediapipe==0.10.8

# TensorFlow with GPU support
!pip install tensorflow[and-cuda]==2.15.0
!pip install tensorflow-hub==0.15.0

# CuPy for GPU array operations (match your CUDA version)
# For CUDA 11.8:
!pip install cupy-cuda11x==12.3.0
# For CUDA 12.x:
# !pip install cupy-cuda12x==12.3.0

# Additional packages
!pip install customtkinter==5.2.1
!pip install pygame==2.5.2
!pip install pydub==0.25.1
!pip install psutil==5.9.6

print("\n" + "="*50)
print("✅ Package installation complete!")
print("="*50)

## 2. CUDA Environment Verification

Verify that GPU is properly detected and CUDA is working.

In [ ]:
import tensorflow as tf
import cv2
import sys

print("="*60)
print("CUDA Environment Verification")
print("="*60)

# Check Python version
print(f"\n📌 Python Version: {sys.version}")

# Check TensorFlow GPU
print(f"\n📌 TensorFlow Version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU(s) Detected: {len(gpus)}")
    for gpu in gpus:
        print(f"   - {gpu}")
        # Get GPU details
        gpu_details = tf.config.experimental.get_device_details(gpu)
        print(f"     Device Name: {gpu_details.get('device_name', 'Unknown')}")
    
    # Test GPU computation
    with tf.device('/GPU:0'):
        a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        b = tf.constant([[5.0, 6.0], [7.0, 8.0]])
        c = tf.matmul(a, b)
    print(f"\n✅ GPU Computation Test: PASSED")
    print(f"   Result: {c.numpy()}")
else:
    print("❌ No GPU detected! Running on CPU.")
    print("   Please install CUDA and cuDNN properly.")

# Check OpenCV CUDA support
print(f"\n📌 OpenCV Version: {cv2.__version__}")
if cv2.cuda.getCudaEnabledDeviceCount() > 0:
    print(f"✅ OpenCV CUDA Support: Enabled")
    print(f"   CUDA Devices: {cv2.cuda.getCudaEnabledDeviceCount()}")
else:
    print("⚠️  OpenCV CUDA Support: Disabled (using CPU fallback)")
    print("   Tip: Install opencv-contrib-python with CUDA build for better performance")

# Check CuPy
try:
    import cupy as cp
    print(f"\n📌 CuPy Version: {cp.__version__}")
    print(f"✅ CuPy CUDA Support: Enabled")
    
    # Test CuPy GPU computation
    x_gpu = cp.array([1, 2, 3, 4, 5])
    y_gpu = x_gpu * 2
    print(f"   CuPy Test: {y_gpu}")
except ImportError:
    print("\n⚠️  CuPy not installed. GPU array operations will use NumPy (CPU).")

# Check MediaPipe
try:
    import mediapipe as mp
    print(f"\n📌 MediaPipe Version: {mp.__version__}")
    print(f"✅ MediaPipe Installed")
except ImportError:
    print("\n❌ MediaPipe not installed!")

print("\n" + "="*60)
print("✅ Environment check complete!")
print("="*60)

## 3. Import Required Libraries

Import all necessary libraries with GPU optimization settings.

In [ ]:
# ========== GPU Optimization Settings ==========
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Reduce TensorFlow logging
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'  # Dynamic GPU memory allocation
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use first GPU

# OpenCV FFMPEG optimization for RTSP streams
if "OPENCV_FFMPEG_CAPTURE_OPTIONS" not in os.environ:
    os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = "rtsp_transport;tcp|stimeout;5000000|video_codec;h264"

# ========== Core Imports ==========
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import face_recognition
from PIL import Image, ImageTk
import time
import json
import csv
import glob
import gc
import psutil
import logging
from logging.handlers import RotatingFileHandler
from datetime import datetime, timedelta
from collections import deque, Counter
from types import SimpleNamespace
import threading
import warnings

# GPU array operations
try:
    import cupy as cp
    GPU_ARRAY_LIB = cp
    USE_CUPY = True
    print("✅ Using CuPy for GPU array operations")
except ImportError:
    GPU_ARRAY_LIB = np
    USE_CUPY = False
    print("⚠️  CuPy not available, using NumPy (CPU fallback)")

# UI Libraries
import tkinter as tk
import customtkinter as ctk
from tkinter import font, simpledialog, messagebox, filedialog

# Audio libraries
try:
    import pygame
    PYGAME_AVAILABLE = True
except ImportError:
    PYGAME_AVAILABLE = False
    print("⚠️  pygame not available (audio alerts disabled)")

try:
    from pydub import AudioSegment
    from pydub.playback import play
    PYDUB_AVAILABLE = True
except ImportError:
    PYDUB_AVAILABLE = False
    print("⚠️  pydub not available (audio alerts disabled)")

# Set appearance
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

# Suppress warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
logging.getLogger('absl').setLevel(logging.ERROR)
logging.getLogger('tensorflow').setLevel(logging.ERROR)

print("\n✅ All libraries imported successfully!")
print(f"   - TensorFlow: {tf.__version__}")
print(f"   - OpenCV: {cv2.__version__}")
print(f"   - MediaPipe: {mp.__version__}")
print(f"   - GPU Acceleration: {'✅ ENABLED' if len(tf.config.list_physical_devices('GPU')) > 0 else '❌ DISABLED'}")

## 4. GPU-Optimized Configuration & Logging

Configure GPU memory management and logging system.

In [ ]:
# ========== GPU Memory Configuration ==========
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth (allocate memory as needed)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Optional: Set memory limit (e.g., 4GB for GTX 1080 with 8GB VRAM)
        # tf.config.set_logical_device_configuration(
        #     gpus[0],
        #     [tf.config.LogicalDeviceConfiguration(memory_limit=4096)]
        # )
        
        print(f"✅ GPU memory growth enabled for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(f"⚠️  GPU configuration error: {e}")

# ========== Configuration Loading ==========
def load_config():
    """Load configuration with GPU-optimized defaults"""
    try:
        config_path = "config.json"
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                return json.load(f)
    except Exception as e:
        print(f"Config load error: {e}. Using GPU-optimized defaults.")
    
    return {
        "detection": {
            "min_detection_confidence": 0.5,
            "min_tracking_confidence": 0.5,
            "face_recognition_tolerance": 0.5,
            "re_detect_interval": 8,
            "use_gpu": True  # ✅ GPU acceleration enabled
        },
        "alert": {
            "default_interval_seconds": 10,
            "alert_cooldown_seconds": 2.5
        },
        "performance": {
            "gui_refresh_ms": 30,
            "pose_buffer_size": 12,
            "frame_skip_interval": 1,  # Lower for GPU (can process more frames)
            "enable_frame_skipping": False,  # Disable with GPU
            "min_buffer_for_classification": 5,
            "gpu_batch_size": 4  # ✅ Process multiple frames in parallel
        },
        "logging": {
            "log_directory": "logs",
            "max_log_size_mb": 10,
            "auto_flush_interval": 50
        },
        "storage": {
            "alert_snapshots_dir": "alert_snapshots",
            "snapshot_retention_days": 30,
            "guard_profiles_dir": "guard_profiles",
            "capture_snapshots_dir": "capture_snapshots",
            "audio_files_dir": "audio_files"
        },
        "monitoring": {
            "mode": "pose",
            "session_restart_prompt_hours": 8
        },
        "gpu": {  # ✅ GPU-specific settings
            "device_id": 0,
            "memory_fraction": 0.8,  # Use 80% of GPU memory
            "allow_growth": True,
            "use_tensorrt": False  # Enable TensorRT optimization (requires conversion)
        }
    }

CONFIG = load_config()

# ========== Logging Setup ==========
os.makedirs(CONFIG["logging"]["log_directory"], exist_ok=True)

logger = logging.getLogger("निराक्षण_CUDA")
logger.setLevel(logging.INFO)

# Console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_formatter = logging.Formatter('%(asctime)s - [GPU] - %(levelname)s - %(message)s')
console_handler.setFormatter(console_formatter)

# File handler
file_handler = RotatingFileHandler(
    os.path.join(CONFIG["logging"]["log_directory"], "session_cuda.log"),
    maxBytes=CONFIG["logging"]["max_log_size_mb"] * 1024 * 1024,
    backupCount=5
)
file_handler.setLevel(logging.INFO)
file_formatter = logging.Formatter('%(asctime)s - [GPU] - %(levelname)s - %(message)s')
file_handler.setFormatter(file_formatter)

logger.addHandler(console_handler)
logger.addHandler(file_handler)

# ========== Create Required Directories ==========
for key in ["alert_snapshots_dir", "guard_profiles_dir", "capture_snapshots_dir", "audio_files_dir"]:
    dir_path = CONFIG["storage"][key]
    os.makedirs(dir_path, exist_ok=True)

# ========== CSV Log File ==========
csv_file = os.path.join(CONFIG["logging"]["log_directory"], "events.csv")
if not os.path.exists(csv_file):
    with open(csv_file, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Name", "Action", "Status", "Image_Path", "Confidence", "GPU_Utilized"])

logger.info("="*60)
logger.info("निराक्षण CUDA System Initialized")
logger.info(f"GPU: {'✅ ENABLED' if CONFIG['detection']['use_gpu'] else '❌ DISABLED'}")
logger.info(f"TensorFlow GPU Devices: {len(tf.config.list_physical_devices('GPU'))}")
logger.info("="*60)

print("\n✅ Configuration and logging initialized!")

## 5. GPU-Accelerated Helper Functions

Utility functions optimized for GPU processing.

In [ ]:
# ========== GPU Memory Management ==========
def optimize_gpu_memory():
    """Optimize GPU memory usage with aggressive garbage collection"""
    try:
        # Clear Python garbage
        gc.collect()
        
        # Clear TensorFlow GPU memory
        tf.keras.backend.clear_session()
        
        # Clear CuPy memory pool if available
        if USE_CUPY:
            mempool = cp.get_default_memory_pool()
            pinned_mempool = cp.get_default_pinned_memory_pool()
            mempool.free_all_blocks()
            pinned_mempool.free_all_blocks()
        
        logger.debug("GPU memory optimized")
    except Exception as e:
        logger.debug(f"GPU memory optimization error: {e}")

def get_gpu_memory_info():
    """Get current GPU memory usage"""
    try:
        if USE_CUPY:
            mempool = cp.get_default_memory_pool()
            used = mempool.used_bytes() / (1024**2)  # MB
            total = mempool.total_bytes() / (1024**2)  # MB
            return {"used_mb": used, "total_mb": total, "utilization": (used/total*100) if total > 0 else 0}
        else:
            return {"used_mb": 0, "total_mb": 0, "utilization": 0}
    except Exception as e:
        logger.debug(f"GPU memory info error: {e}")
        return {"used_mb": 0, "total_mb": 0, "utilization": 0}

# ========== GPU-Accelerated Image Processing ==========
def gpu_resize_frame(frame, target_size, use_gpu=True):
    """Resize frame using GPU if available"""
    if use_gpu and cv2.cuda.getCudaEnabledDeviceCount() > 0:
        try:
            # Upload to GPU
            gpu_frame = cv2.cuda_GpuMat()
            gpu_frame.upload(frame)
            
            # Resize on GPU
            gpu_resized = cv2.cuda.resize(gpu_frame, target_size)
            
            # Download from GPU
            return gpu_resized.download()
        except Exception as e:
            logger.debug(f"GPU resize failed, using CPU: {e}")
    
    # CPU fallback
    return cv2.resize(frame, target_size)

def gpu_cvtColor(frame, conversion_code, use_gpu=True):
    """Convert color space using GPU if available"""
    if use_gpu and cv2.cuda.getCudaEnabledDeviceCount() > 0:
        try:
            gpu_frame = cv2.cuda_GpuMat()
            gpu_frame.upload(frame)
            gpu_converted = cv2.cuda.cvtColor(gpu_frame, conversion_code)
            return gpu_converted.download()
        except Exception as e:
            logger.debug(f"GPU cvtColor failed, using CPU: {e}")
    
    return cv2.cvtColor(frame, conversion_code)

def gpu_apply_clahe(frame, use_gpu=True):
    """Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) using GPU"""
    if use_gpu and cv2.cuda.getCudaEnabledDeviceCount() > 0:
        try:
            # Convert to LAB color space
            lab = gpu_cvtColor(frame, cv2.COLOR_BGR2LAB, use_gpu=True)
            l_channel, a_channel, b_channel = cv2.split(lab)
            
            # Upload L channel to GPU
            gpu_l = cv2.cuda_GpuMat()
            gpu_l.upload(l_channel)
            
            # Apply CLAHE on GPU
            clahe = cv2.cuda.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
            gpu_l_clahe = clahe.apply(gpu_l, cv2.cuda_Stream.Null())
            
            # Download result
            l_clahe = gpu_l_clahe.download()
            
            # Merge back
            lab_clahe = cv2.merge([l_clahe, a_channel, b_channel])
            return gpu_cvtColor(lab_clahe, cv2.COLOR_LAB2BGR, use_gpu=True)
        except Exception as e:
            logger.debug(f"GPU CLAHE failed, using CPU: {e}")
    
    # CPU fallback
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l_channel)
    lab_clahe = cv2.merge([l_clahe, a_channel, b_channel])
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

# ========== File Storage Utilities ==========
def get_storage_paths():
    """Get organized storage directory paths"""
    paths = {
        "guard_profiles": CONFIG["storage"]["guard_profiles_dir"],
        "capture_snapshots": CONFIG["storage"]["capture_snapshots_dir"],
        "logs": CONFIG["logging"]["log_directory"],
        "alert_snapshots": CONFIG["storage"]["alert_snapshots_dir"]
    }
    
    for path in paths.values():
        os.makedirs(path, exist_ok=True)
    
    return paths

def save_guard_face(face_image, guard_name, angle="front"):
    """Save guard face image"""
    paths = get_storage_paths()
    safe_name = guard_name.strip().replace(" ", "_")
    
    if angle and angle != "front":
        guard_dir = os.path.join(paths["guard_profiles"], f"target_{safe_name}")
        os.makedirs(guard_dir, exist_ok=True)
        profile_path = os.path.join(guard_dir, f"{angle}.jpg")
    else:
        profile_path = os.path.join(paths["guard_profiles"], f"target_{safe_name}_face.jpg")
        guard_dir = os.path.join(paths["guard_profiles"], f"target_{safe_name}")
        os.makedirs(guard_dir, exist_ok=True)
        front_path = os.path.join(guard_dir, "front.jpg")
        cv2.imwrite(front_path, face_image)
    
    cv2.imwrite(profile_path, face_image)
    logger.info(f"Saved guard face ({angle}): {profile_path}")
    return profile_path

def cleanup_old_snapshots():
    """Cleanup old alert snapshots"""
    try:
        retention_days = CONFIG["storage"]["snapshot_retention_days"]
        cutoff_time = datetime.now() - timedelta(days=retention_days)
        snapshot_dir = CONFIG["storage"]["alert_snapshots_dir"]
        
        for filename in os.listdir(snapshot_dir):
            filepath = os.path.join(snapshot_dir, filename)
            if os.path.isfile(filepath):
                file_time = datetime.fromtimestamp(os.path.getmtime(filepath))
                if file_time < cutoff_time:
                    os.remove(filepath)
    except Exception as e:
        logger.error(f"Snapshot cleanup error: {e}")

# Start cleanup thread
threading.Thread(target=cleanup_old_snapshots, daemon=True).start()

print("\n✅ GPU-optimized helper functions loaded!")
print(f"   - GPU Resize: {'✅ Available' if cv2.cuda.getCudaEnabledDeviceCount() > 0 else '❌ Not Available'}")
print(f"   - GPU CLAHE: {'✅ Available' if cv2.cuda.getCudaEnabledDeviceCount() > 0 else '❌ Not Available'}")
print(f"   - CuPy Arrays: {'✅ Enabled' if USE_CUPY else '❌ Disabled'}")

## 6. GPU-Accelerated Face Detection & Recognition

Face detection and recognition with CUDA acceleration.

In [ ]:
# ========== GPU-Accelerated Face Detection ==========
def detect_faces_gpu(rgb_frame, model="hog", use_gpu=True):
    """
    Detect faces using GPU-accelerated face_recognition library.
    
    Note: face_recognition with dlib compiled with CUDA support will automatically
    use GPU for HOG-based detection. For CNN model, GPU is always used if available.
    """
    try:
        # face_recognition will use GPU if dlib was compiled with CUDA
        if use_gpu and model == "cnn":
            # CNN model is always GPU-accelerated if CUDA is available
            face_locations = face_recognition.face_locations(rgb_frame, model="cnn")
        else:
            # HOG model (faster, uses GPU if dlib has CUDA support)
            face_locations = face_recognition.face_locations(rgb_frame, model="hog")
        
        return face_locations
    except Exception as e:
        logger.debug(f"Face detection error: {e}")
        return []

def generate_face_encodings_gpu(rgb_frame, face_locations, num_jitters=1):
    """
    Generate face encodings using GPU if available.
    
    face_recognition will use GPU for encoding if dlib was compiled with CUDA.
    """
    try:
        # face_recognition will automatically use GPU if available
        encodings = face_recognition.face_encodings(
            rgb_frame, 
            known_face_locations=face_locations,
            num_jitters=num_jitters
        )
        return encodings
    except Exception as e:
        logger.debug(f"Face encoding error: {e}")
        return []

def compare_faces_gpu(known_encodings, face_encoding, tolerance=0.6):
    """
    Compare face encodings using GPU-accelerated distance calculation.
    """
    try:
        if USE_CUPY and len(known_encodings) > 0:
            # Convert to CuPy arrays for GPU computation
            known_arr = cp.array(known_encodings)
            face_arr = cp.array(face_encoding)
            
            # Compute Euclidean distances on GPU
            distances = cp.linalg.norm(known_arr - face_arr, axis=1)
            
            # Convert back to CPU for comparison
            matches = distances.get() <= tolerance
            return matches.tolist()
        else:
            # CPU fallback using face_recognition
            return face_recognition.compare_faces(known_encodings, face_encoding, tolerance=tolerance)
    except Exception as e:
        logger.debug(f"Face comparison error: {e}")
        return [False] * len(known_encodings)

def face_distance_gpu(face_encodings, face_to_compare):
    """
    Calculate face distances using GPU.
    """
    try:
        if USE_CUPY and len(face_encodings) > 0:
            encodings_arr = cp.array(face_encodings)
            compare_arr = cp.array(face_to_compare)
            distances = cp.linalg.norm(encodings_arr - compare_arr, axis=1)
            return distances.get()
        else:
            return face_recognition.face_distance(face_encodings, face_to_compare)
    except Exception as e:
        logger.debug(f"Face distance error: {e}")
        return np.array([1.0] * len(face_encodings))

# Test GPU face detection
print("\n✅ GPU-accelerated face detection functions loaded!")
print("   Testing face detection capabilities...")

# Create a dummy test image
test_img = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
test_rgb = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

# Test face detection (should return empty list for random image)
start_time = time.time()
faces = detect_faces_gpu(test_rgb, model="hog", use_gpu=True)
elapsed = time.time() - start_time

print(f"   Face detection test: {elapsed*1000:.2f}ms")
print(f"   Detected faces: {len(faces)} (expected 0 for random image)")

## 7. GPU-Accelerated Pose Estimation

MediaPipe Pose with GPU backend for real-time pose tracking.

In [ ]:
# ========== Initialize MediaPipe Pose with GPU ==========
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Initialize pose detector with GPU delegate
pose_detector_gpu = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,  # 0=Lite, 1=Full, 2=Heavy
    smooth_landmarks=True,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

logger.info("MediaPipe Pose initialized with GPU backend")

# ========== Pose Classification Function ==========
def classify_action(landmarks, h, w):
    """
    Classify pose action from MediaPipe landmarks.
    Supports: Hands Up, Hands Crossed, One Hand Raised, T-Pose, Sit, Standing
    """
    try:
        NOSE = mp_holistic.PoseLandmark.NOSE.value
        L_WRIST = mp_holistic.PoseLandmark.LEFT_WRIST.value
        R_WRIST = mp_holistic.PoseLandmark.RIGHT_WRIST.value
        L_ELBOW = mp_holistic.PoseLandmark.LEFT_ELBOW.value
        R_ELBOW = mp_holistic.PoseLandmark.RIGHT_ELBOW.value
        L_SHOULDER = mp_holistic.PoseLandmark.LEFT_SHOULDER.value
        R_SHOULDER = mp_holistic.PoseLandmark.RIGHT_SHOULDER.value
        L_HIP = mp_holistic.PoseLandmark.LEFT_HIP.value
        R_HIP = mp_holistic.PoseLandmark.RIGHT_HIP.value
        L_KNEE = mp_holistic.PoseLandmark.LEFT_KNEE.value
        R_KNEE = mp_holistic.PoseLandmark.RIGHT_KNEE.value

        nose = landmarks[NOSE]
        l_wrist = landmarks[L_WRIST]
        r_wrist = landmarks[R_WRIST]
        l_shoulder = landmarks[L_SHOULDER]
        r_shoulder = landmarks[R_SHOULDER]
        l_hip = landmarks[L_HIP]
        r_hip = landmarks[R_HIP]
        l_knee = landmarks[L_KNEE]
        r_knee = landmarks[R_KNEE]

        # Calculate body scale for distance-agnostic detection
        shoulder_to_hip_dist = abs(l_shoulder.y - l_hip.y)
        if shoulder_to_hip_dist < 0.01:
            return "Standing"
        
        # Threshold multipliers based on body size
        HANDS_UP_THRESHOLD = shoulder_to_hip_dist * 0.4
        HANDS_CROSSED_TOLERANCE = shoulder_to_hip_dist * 0.3
        
        # Check visibility
        l_wrist_visible = l_wrist.visibility > 0.45
        r_wrist_visible = r_wrist.visibility > 0.45
        nose_visible = nose.visibility > 0.40
        
        # Hands Up Detection
        if l_wrist_visible and r_wrist_visible and nose_visible:
            wrist_above_nose_l = (nose.y - l_wrist.y) > (HANDS_UP_THRESHOLD * 0.8)
            wrist_above_nose_r = (nose.y - r_wrist.y) > (HANDS_UP_THRESHOLD * 0.8)
            
            if wrist_above_nose_l and wrist_above_nose_r:
                return "Hands Up"
        
        # One Hand Raised Detection
        if l_wrist_visible and r_wrist_visible:
            chest_y = (l_shoulder.y + r_shoulder.y) / 2
            
            left_raised = (nose.y - l_wrist.y) > (HANDS_UP_THRESHOLD * 0.8)
            right_down = (r_wrist.y - chest_y) > (HANDS_CROSSED_TOLERANCE * 0.8)
            
            if left_raised and right_down:
                return "One Hand Raised (Left)"
            
            right_raised = (nose.y - r_wrist.y) > (HANDS_UP_THRESHOLD * 0.8)
            left_down = (l_wrist.y - chest_y) > (HANDS_CROSSED_TOLERANCE * 0.8)
            
            if right_raised and left_down:
                return "One Hand Raised (Right)"
        
        # Sit/Stand Detection
        if l_knee.visibility > 0.30 and r_knee.visibility > 0.30:
            thigh_angle_l = abs(l_knee.y - l_hip.y)
            thigh_angle_r = abs(r_knee.y - r_hip.y)
            avg_thigh_angle = (thigh_angle_l + thigh_angle_r) / 2
            
            sit_threshold = shoulder_to_hip_dist * 0.15
            
            if avg_thigh_angle < sit_threshold:
                return "Sit"
            else:
                return "Standing"
        
        return "Standing"

    except Exception as e:
        logger.debug(f"Pose classification error: {e}")
        return "Unknown"

def process_pose_gpu(frame, use_gpu=True):
    """
    Process frame for pose detection using GPU.
    Returns pose landmarks and action classification.
    """
    try:
        # Convert BGR to RGB
        rgb_frame = gpu_cvtColor(frame, cv2.COLOR_BGR2RGB, use_gpu=use_gpu)
        
        # Process with MediaPipe (automatically uses GPU if available)
        results = pose_detector_gpu.process(rgb_frame)
        
        if results.pose_landmarks:
            h, w, _ = frame.shape
            landmarks = results.pose_landmarks.landmark
            
            # Classify action
            action = classify_action(landmarks, h, w)
            
            return {
                "landmarks": landmarks,
                "action": action,
                "confidence": np.mean([lm.visibility for lm in landmarks]),
                "pose_world_landmarks": results.pose_world_landmarks
            }
        
        return None
    except Exception as e:
        logger.debug(f"Pose processing error: {e}")
        return None

def draw_pose_landmarks_gpu(frame, pose_result):
    """
    Draw pose landmarks on frame.
    """
    try:
        if pose_result and pose_result.get("landmarks"):
            # Create pose landmarks object for drawing
            landmarks_proto = mp_pose.PoseLandmark()
            for landmark in pose_result["landmarks"]:
                landmarks_proto.landmark.add().CopyFrom(landmark)
            
            # Draw landmarks
            mp_drawing.draw_landmarks(
                frame,
                landmarks_proto,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )
    except Exception as e:
        logger.debug(f"Draw landmarks error: {e}")

print("\n✅ GPU-accelerated pose estimation loaded!")
print("   MediaPipe Pose initialized with GPU backend")

# Test pose detection
test_frame = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
start_time = time.time()
pose_result = process_pose_gpu(test_frame, use_gpu=True)
elapsed = time.time() - start_time

print(f"   Pose detection test: {elapsed*1000:.2f}ms")
print(f"   Result: {pose_result is not None}")